In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
import time


In [5]:
url = 'http://192.168.1.5:5000'
volume = {'volume':5}

In [6]:
r = requests.post(url + '/get_status', json = {'name':'10cc'})

In [7]:
r.content

b'{"remaining_volume":null,"syringe_loaded":false}\n'

In [8]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [80]:
jubilee.home_all()

In [9]:
deck = jubilee.load_deck('lab_automation_deck.json')

# load a well plate 

In [10]:
samples = jubilee.load_labware('falcon_48_wellplate_1500ul.json', 0)
samples.manual_offset([(21.4, 82.9), (134.1, 83.2), (133.9, 12.8)])

New manual offset applied to falcon_48_wellplate_1500ul


In [11]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
#stocks.manual_offset()

In [12]:
syringe_10 = syringe.HTTPSyringe.from_config(3, "/home/brendenpelkie/Code/science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")

In [21]:
syringe_1 = syringe.HTTPSyringe.from_config(2, "/home/brendenpelkie/Code/science-jubilee/src/science_jubilee/tools/configs/1cc_syringe.json")

In [25]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1)

In [15]:
syringe_1.status()

{'remaining_volume': None, 'syringe_loaded': False}

In [16]:
syringe_1.name

'1cc'

In [17]:
syringe_1.set_pulsewidth(1500)

In [18]:
syringe_1.load_syringe(560, 1500)

Loaded syringe, remaining volume 560 uL


In [79]:
jubilee.park_tool()

In [78]:
jubilee.pickup_tool(syringe_1)

## Manual syringe load:

In [77]:
syringe_1.set_pulsewidth(syringe_1.empty_position-1)

In [73]:
aspirate = samples[0].bottom(+200)
aspirate_vol = 995 - syringe_1.remaining_volume
#aspirate_vol = 9995
syringe_1.aspirate(aspirate_vol, aspirate)

In [74]:
syringe_1.set_pulsewidth(1280)

In [75]:
## Run dispense experiment 

In [76]:
# time step sequence:
"""
3 samples per each:
- 1
- 10
- 50
- 100
- 200
- 300
"""

#### Modify here
test_step = 100

######Don't touch here ######
syringe_1.set_pulsewidth(1300)
syringe_pulsewidth = 1300

jubilee.pickup_tool(syringe_1)
for i in range(1):
    dispense_location = samples[i].bottom(+55)
    new_pulsewidth = syringe_pulsewidth + test_step
    syringe_pulsewidth = new_pulsewidth

    jubilee.move_to(x = dispense_location.point[0], y = dispense_location.point[1], wait = True)
    jubilee.move_to(z = dispense_location.point[2], wait = True)
    syringe_1.set_pulsewidth(new_pulsewidth)
    time.sleep(10)


In [28]:
dispense_location.point

(176.41, 170.79, 67.5)